In [1]:
from pettingzoo.sisl import waterworld_v4
import supersuit as ss
from tianshou.env.pettingzoo_env import PettingZooEnv
import numpy as np
import torch

def get_env_info():
    """获取详细的环境信息"""
    print("=== 环境信息调试 ===\n")
    
    # 1. 创建原始环境
    agent_algos = ["DQN", "DQN", "DQN", "DQN"]
    raw_env = waterworld_v4.env(
        render_mode=None,
        n_predators=2,
        n_preys=2,
        n_evaders=50,
        n_obstacles=2,
        obstacle_coord=[(0.2, 0.2), (0.8, 0.2)],
        n_poisons=2,
        agent_algorithms=agent_algos
    )
    
    # 2. 应用supersuit包装
    wrapped_env = ss.black_death_v3(raw_env)
    
    # 3. 创建Tianshou环境
    tianshou_env = PettingZooEnv(wrapped_env)
    
    print("1. 基本环境信息:")
    print(f"   - 环境类型: {type(tianshou_env)}")
    print(f"   - 智能体列表: {tianshou_env.agents}")
    print(f"   - 智能体数量: {len(tianshou_env.agents)}")
    
    print("\n2. 观测空间信息:")
    obs_space = tianshou_env.observation_space
    print(f"   - 观测空间类型: {type(obs_space)}")
    print(f"   - 观测空间: {obs_space}")
    
    if hasattr(obs_space, 'shape'):
        print(f"   - 观测空间形状: {obs_space.shape}")
    elif hasattr(obs_space, 'spaces'):
        print(f"   - 观测空间字典: {obs_space.spaces}")
        for key, space in obs_space.spaces.items():
            print(f"     - {key}: {space} (shape: {getattr(space, 'shape', 'N/A')})")
    
    print("\n3. 动作空间信息:")
    action_space = tianshou_env.action_space
    print(f"   - 动作空间类型: {type(action_space)}")
    print(f"   - 动作空间: {action_space}")
    if hasattr(action_space, 'shape'):
        print(f"   - 动作空间形状: {action_space.shape}")
        print(f"   - 动作空间大小: {action_space.shape or action_space.n}")
    
    print("\n4. 重置环境并获取初始观测:")
    obs, info = tianshou_env.reset()
    
    print(f"   - 重置后观测类型: {type(obs)}")
    if isinstance(obs, dict):
        print("   - 观测字典内容:")
        for key, value in obs.items():
            print(f"     - {key}: {type(value)} {getattr(value, 'shape', '')} {getattr(value, 'dtype', '')}")
            if isinstance(value, np.ndarray) and value.size <= 10:
                print(f"       样例值: {value}")
    else:
        print(f"   - 观测形状: {getattr(obs, 'shape', 'N/A')}")
        print(f"   - 观测数据类型: {getattr(obs, 'dtype', 'N/A')}")
        if hasattr(obs, 'shape') and obs.size <= 20:
            print(f"   - 样例观测值: {obs}")
    
    print(f"   - 重置信息: {info}")
    
    print("\n5. 执行一步动作:")
    # 随机选择动作
    if hasattr(action_space, 'sample'):
        action = action_space.sample()
    else:
        action = np.random.randint(0, action_space.n)
    
    print(f"   - 选择的动作: {action}")
    
    try:
        next_obs, reward, done, truncated, next_info = tianshou_env.step(action)
        print(f"   - 步骤后观测类型: {type(next_obs)}")
        print(f"   - 奖励: {reward}")
        print(f"   - 完成状态: {done}")
        print(f"   - 截断状态: {truncated}")
        
        if isinstance(next_obs, dict):
            print("   - 下一步观测字典:")
            for key, value in next_obs.items():
                print(f"     - {key}: {type(value)} {getattr(value, 'shape', '')} {getattr(value, 'dtype', '')}")
        
    except Exception as e:
        print(f"   - 步骤执行错误: {e}")
    
    print("\n6. 数据类型检查:")
    # 检查观测数据是否为numpy数组还是tensor
    if isinstance(obs, dict):
        for key, value in obs.items():
            print(f"   - {key}:")
            print(f"     类型: {type(value)}")
            print(f"     是否为numpy数组: {isinstance(value, np.ndarray)}")
            print(f"     是否为tensor: {isinstance(value, torch.Tensor)}")
            if hasattr(value, 'device'):
                print(f"     设备: {value.device}")
    else:
        print(f"   - 观测数据:")
        print(f"     类型: {type(obs)}")
        print(f"     是否为numpy数组: {isinstance(obs, np.ndarray)}")
        print(f"     是否为tensor: {isinstance(obs, torch.Tensor)}")
    
    tianshou_env.close()
    
    return {
        'obs_space': obs_space,
        'action_space': action_space,
        'agents': tianshou_env.agents,
        'sample_obs': obs
    }

if __name__ == "__main__":
    env_info = get_env_info()

/home/qrbao/anaconda3/envs/rllib_discret/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-03 05:00:38,134	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


=== 环境信息调试 ===

1. 基本环境信息:
   - 环境类型: <class 'tianshou.env.pettingzoo_env.PettingZooEnv'>
   - 智能体列表: ['predator_0', 'predator_1', 'prey_0', 'prey_1']
   - 智能体数量: 4

2. 观测空间信息:
   - 观测空间类型: <class 'gymnasium.spaces.box.Box'>
   - 观测空间: Box(-inf, inf, (302,), float32)
   - 观测空间形状: (302,)

3. 动作空间信息:
   - 动作空间类型: <class 'gymnasium.spaces.discrete.Discrete'>
   - 动作空间: Discrete(9)
   - 动作空间形状: ()
   - 动作空间大小: 9

4. 重置环境并获取初始观测:
   - 重置后观测类型: <class 'dict'>
   - 观测字典内容:
     - agent_id: <class 'str'>  
     - obs: <class 'numpy.ndarray'> (302,) float32
     - mask: <class 'list'>  
   - 重置信息: {}

5. 执行一步动作:
   - 选择的动作: 7
   - 步骤后观测类型: <class 'dict'>
   - 奖励: [0, 0, 0, 0]
   - 完成状态: False
   - 截断状态: False
   - 下一步观测字典:
     - agent_id: <class 'str'>  
     - obs: <class 'numpy.ndarray'> (302,) float32
     - mask: <class 'list'>  

6. 数据类型检查:
   - agent_id:
     类型: <class 'str'>
     是否为numpy数组: False
     是否为tensor: False
   - obs:
     类型: <class 'numpy.ndarray'>
     是否为numpy数组: True
  

In [2]:
import pymunk

class Agent:
    def __init__(self, space, x, y):
        # 创建物理身体
        self.body = pymunk.Body(1, 1666)  # 质量和惯性
        self.body.position = (x, y)
        
        # 创建形状并添加到物理世界
        self.shape = pymunk.Circle(self.body, 15)  # 半径15的圆形
        self.shape.elasticity = 0.8  # 弹性
        space.add(self.body, self.shape)
        
        # 添加到物体列表（例如所有agents）
        self.alive = True

    def kill(self, space):
        if self.alive:
            # 销毁物理对象
            space.remove(self.body, self.shape)
            self.body = None
            self.shape = None
            self.alive = False
            print("Agent has been killed and removed from the world.")

# 示例用法
space = pymunk.Space()
agent = Agent(space, 100, 100)



In [3]:
# 假设某些条件触发了agent的死亡
agent.kill(space)

Agent has been killed and removed from the world.


AttributeError: 'WaterworldBase' object has no attribute 'num_agents'